查看显卡信息

In [ ]:
!nvidia-smi

# 一般笔记本电脑的可用GPU都只有1个，如果需要尝试使用多个GPU，可以去租用算力平台查看

Sun Apr 20 19:17:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.07                 Driver Version: 566.07         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   53C    P8              3W /   80W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

可以指定计算设备

在PyTorch中，CPU和GPU可以用torch.device('cpu') 和torch.device('cuda')表示。 
应该注意的是，cpu设备意味着所有物理CPU和内存，这意味着PyTorch的计算将尝试使用所有CPU核心。 
然而，gpu设备只代表一个卡和相应的显存。 

如果有多个GPU，我们使用torch.device(f'cuda:{i}') 来表示第i块GPU（从0开始）。 另外，cuda:0和cuda是等价的。

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

注意这样的操作只是创建了设备对象，并没有指定模型运行的设备

查询可用的GPU数量

In [3]:
torch.cuda.device_count()

1

In [4]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

张量和GPU

可以查询张量所在的设备，默认情况下张量在CPU上创建


需要注意的是，无论何时我们要对多个项进行操作，它们都必须在同一个设备上。例如，如果我们对两个张量求和，我们需要确保两个张量都位于同一个设备上，否则框架将不知道在哪里存储结果，甚至不知道在哪里执行计算。

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

存储在GPU上

有几种方法可以在GPU上存储张量。 
例如，我们可以在创建张量时指定存储设备。接下来，我们在第一个gpu上创建张量变量X。在GPU上创建的张量只消耗这个GPU的显存。我们可以使用nvidia-smi命令查看显存使用情况。 
一般来说，我们需要确保不创建超过GPU显存限制的数据。

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [7]:
# 尝试在另一个GPU上创建张量
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.7430, 0.3348, 0.6747],
        [0.0443, 0.5402, 0.6470]])

如果要计算X+Y则需要考虑在哪里执行这个操作

不要简单地X加上Y，因为这会导致异常，运行时引擎不知道该怎么做：它在同一设备上找不到数据会导致失败。 
如果Y位于第二个GPU上，则我们需要将X移到那里，然后才能执行相加运算

In [8]:
Z = Y.cuda(0)
print(Y)
print(Z)

tensor([[0.7430, 0.3348, 0.6747],
        [0.0443, 0.5402, 0.6470]])
tensor([[0.7430, 0.3348, 0.6747],
        [0.0443, 0.5402, 0.6470]], device='cuda:0')


当数据都在同一个GPU上后，可以进行相加

In [9]:
X + Z

tensor([[1.7430, 1.3348, 1.6747],
        [1.0443, 1.5402, 1.6470]], device='cuda:0')

人们使用GPU来进行机器学习，因为单个GPU相对运行速度快。 
但是在设备（CPU、GPU和其他机器）之间传输数据比计算慢得多。这也使得并行化变得更加困难，因为我们必须等待数据被发送（或者接收），然后才能继续进行更多的操作。 
这就是为什么拷贝操作要格外小心。 
根据经验，多个小操作比一个大操作糟糕得多。 
此外，一次执行几个操作比代码中散布的许多单个操作要好得多。如果一个设备必须等待另一个设备才能执行其他操作，那么这样的操作可能会阻塞。

最后，当我们打印张量或将张量转换为NumPy格式时，如果数据不在内存中，框架会首先将其复制到内存中，这会导致额外的传输开销。更糟糕的是，它现在受制于全局解释器锁，使得一切都得等待Python完成。

类似地，神经网络模型可以指定设备

下面的代码将模型参数放在GPU上

In [10]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

当输入为GPU上的张量时，模型将在同一GPU上计算结果。

In [11]:
net(X)

tensor([[-0.2459],
        [-0.2459]], device='cuda:0', grad_fn=<AddmmBackward0>)

确认模型参数存储在同一个GPU上

In [12]:
net[0].weight.data.device

device(type='cuda', index=0)